In [ ]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit.quantum_info.operators import Operator
from qiskit.circuit.library import UnitaryGate, QFT
from qiskit_aer import AerSimulator
import numpy as np
from math import gcd, floor, log
from fractions import Fraction
import random
import time
import pandas as pd
import matplotlib.pyplot as plt
import os

# Ensure result directory exists
os.makedirs("result", exist_ok=True)

MAX_ATTEMPTS = 20

def mod_mult_gate(b, N):
    n = floor(log(N - 1, 2)) + 1
    U = np.zeros((2 ** n, 2 ** n))
    for x in range(N):
        U[b * x % N][x] = 1
    for x in range(N, 2 ** n):
        U[x][x] = 1
    if not np.allclose(U.conj().T @ U, np.eye(U.shape[0])):
        raise ValueError(f"Generated U matrix for b={b}, N={N} is not unitary!")
    return UnitaryGate(U, label=f"M_{b}")

def order_finding_circuit(a, N):
    n = floor(log(N - 1, 2)) + 1
    m = 2 * n
    control = QuantumRegister(m, name="X")
    target = QuantumRegister(n, name="Y")
    output = ClassicalRegister(m, name="Z")
    qc = QuantumCircuit(control, target, output)
    qc.x(target[0])  # |1> in target register

    for k, qubit in enumerate(control):
        qc.h(qubit)
        b = pow(a, 2 ** k, N)
        qc.compose(mod_mult_gate(b, N).control(), [qubit] + list(target), inplace=True)

    qc.compose(QFT(m, inverse=True), qubits=control, inplace=True)
    qc.measure(control, output)
    return qc

def find_order_quantum(a, N):
    n = floor(log(N - 1, 2)) + 1
    m = 2 * n
    qc = order_finding_circuit(a, N)
    transpiled = transpile(qc, AerSimulator())
    while True:
        result = AerSimulator().run(transpiled, shots=1, memory=True).result()
        y = int(result.get_memory()[0], 2)
        r = Fraction(y / 2 ** m).limit_denominator(N).denominator
        if r == 0:
            continue
        if pow(a, r, N) == 1:
            return r, qc

N_list = [15, 21, 33, 35, 39, 51, 55, 57, 85, 91, 93, 95, 111]
quantum_results = []

for N in N_list:
    print(f"\nProcessing N = {N} (quantum only)...")
    q_start = time.time()
    q_attempts = 0
    FACTOR_FOUND = False
    q_factor = q_elapsed = q_depth = q_width = q_gates = -1

    while not FACTOR_FOUND and q_attempts < MAX_ATTEMPTS:
        q_attempts += 1
        a = random.choice([x for x in range(2, N) if gcd(x, N) == 1])
        print(f"  Attempt {q_attempts}: Trying a = {a}")
        try:
            r, qc = find_order_quantum(a, N)
            print(f"    Order found: r = {r}")
            if r % 2 == 0:
                x = pow(a, r // 2, N) - 1
                d = gcd(x, N)
                print(f"    r even, gcd(a^(r/2)-1, N) = {d}")
                if d > 1:
                    q_elapsed = time.time() - q_start
                    q_depth = qc.depth()
                    q_width = qc.width()
                    q_gates = qc.size()
                    q_factor = d
                    FACTOR_FOUND = True
                    print(f"    Success: factor = {q_factor}, time = {q_elapsed:.2f}s, depth = {q_depth}, width = {q_width}, gates = {q_gates}")
                else:
                    print(f"    Failed: gcd result = 1 → retry")
            else:
                print(f"    r is odd → retry")
        except Exception as e:
            print(f"    Exception for N={N}, a={a}: {e}")
            continue

        # Update CSV after each attempt
        quantum_results.append({
            "N": N, "factor": q_factor, "time": q_elapsed,
            "depth": q_depth, "width": q_width, "gates": q_gates, "attempts": q_attempts
        })
        pd.DataFrame(quantum_results).to_csv("result/shor_quantum_results.csv", index=False)

    if not FACTOR_FOUND:
        print(f"Failed to find factor for N={N} after {MAX_ATTEMPTS} attempts.")

# Final CSV and plots
df_quantum = pd.DataFrame(quantum_results)
df_quantum.to_csv("result/shor_quantum_results.csv", index=False)

plt.figure(figsize=(10, 6))
plt.plot(df_quantum['N'], df_quantum['time'], marker='o', label='Quantum Time')
plt.xlabel("Composite Number N")
plt.ylabel("Execution Time (seconds)")
plt.title("Quantum Execution Time")
plt.legend()
plt.grid(True)
plt.savefig("result/quantum_execution_time.png")
plt.close()

plt.figure(figsize=(10, 6))
plt.plot(df_quantum['N'], df_quantum['attempts'], marker='o', label='Quantum Attempts')
plt.xlabel("Composite Number N")
plt.ylabel("Number of Attempts")
plt.title("Quantum Attempts Until Success")
plt.legend()
plt.grid(True)
plt.savefig("result/quantum_attempts.png")
plt.close()

plt.figure(figsize=(10, 6))
plt.plot(df_quantum['N'], df_quantum['gates'], marker='^', color='orange')
plt.xlabel("Composite Number N")
plt.ylabel("Gate Count")
plt.title("Quantum Circuit Gate Count")
plt.grid(True)
plt.savefig("result/quantum_gate_count.png")
plt.close()
